In [1]:
import pyspark
from pyspark.sql.functions import udf, col
from pyspark.sql.types import IntegerType, StringType, DoubleType
from pyspark.ml import Transformer, Estimator, Pipeline
from pyspark.ml.classification import LogisticRegression
from mmlspark import *

import numpy as np, pandas as pd, os, sys, time
from os.path import join, abspath, exists

In [2]:
model = ModelDownloader(spark, "/models").downloadByName("ResNet50")

In [3]:
model = ModelDownloader(spark, "models").downloadByName("ResNet50")

In [4]:
# Load the images
# use flowers_and_labels.parquet on larger cluster in order to get better results
imagesWithLabels = spark.read.parquet("wasbs://publicwasb@mmlspark.blob.core.windows.net/flowers_and_labels_small.parquet") \
  .withColumn("labels", col("labels").cast("Double"))

imagesWithLabels.printSchema()

![Smiley face](https://i.imgur.com/p2KgdYL.jpg)

In [6]:
# Make some featurizers
it = ImageTransformer()\
    .setOutputCol("scaled")\
    .resize(height = 60, width = 60)

ur = UnrollImage()\
    .setInputCol("scaled")\
    .setOutputCol("features")
    
dc1 = DropColumns().setCols(["scaled", "image"])

lr1 = LogisticRegression().setFeaturesCol("features").setLabelCol("labels")

dc2 = DropColumns().setCols(["features"])

basicModel = Pipeline(stages=[it, ur, dc1, lr1, dc2])

In [7]:
resnet = ImageFeaturizer()\
    .setInputCol("image")\
    .setOutputCol("features")\
    .setModelLocation(spark, model.uri)\
    .setLayerNames(model.layerNames)\
    .setCutOutputLayers(1)
    
dc3 = DropColumns().setCols(["image"])
    
lr2 = LogisticRegression().setFeaturesCol("features").setLabelCol("labels")

dc4 = DropColumns().setCols(["features"])

deepModel = Pipeline(stages=[resnet, dc3, lr2, dc4])    

![Resnet 18](https://i.imgur.com/Mb4Dyou.png)

### How does it work?

![Convolutional network weights](http://i.stack.imgur.com/Hl2H6.png)

### Run the experiment

In [11]:
def timedExperiment(model, train, test):
  start = time.time()
  result =  model.fit(train).transform(test).toPandas()
  print("Experiment took {}s".format(time.time() - start))
  return result

In [12]:
train, test = imagesWithLabels.randomSplit([.8,.2])
train.count(), test.count()

In [13]:
basicResults = timedExperiment(basicModel, train, test)

In [14]:
deepResults = timedExperiment(deepModel, train, test)

### Plot confusion matrix.

In [16]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import pandas as pd
from glob import glob
import numpy as np
def evaluate(results, name):
    y, y_hat = results["labels"],results["prediction"]
    y = [int(l) for l in y]

    accuracy = np.mean([1. if pred==true else 0. for (pred,true) in zip(y_hat,y)])
    cm = confusion_matrix(y, y_hat)
    cm = cm.astype("float") / cm.sum(axis=1)[:, np.newaxis]

    plt.text(40, 10,"$Accuracy$ $=$ ${}\%$".format(round(accuracy*100,1)),fontsize=14)
    plt.imshow(cm, interpolation="nearest", cmap=plt.cm.Blues)
    plt.colorbar()
    plt.xlabel("$Predicted$ $label$", fontsize=18)
    plt.ylabel("$True$ $Label$", fontsize=18)
    plt.title("$Normalized$ $CM$ $for$ ${}$".format(name))

plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
evaluate(deepResults,"CNTKModel + LR")
plt.subplot(1,2,2)
evaluate(basicResults,"LR")
display(plt.show())